# Robotic Systems I (ECE-DK808)

## Electrical and Computer Engineering Department, University of Patras, Greece

**Instructor:** Konstantinos Chatzilygeroudis (costashatz@upatras.gr)

## Lab 8

### Inverse Kinematics via Quadratic Programming (QP)

We would like to solve the inverse kinematics for a KUKA manipulator using a QP solver! The high-level steps are as follows:

- At each iteration $k$, we solve a QP instead of the pseudoinverse
  1. $\boldsymbol{q}_k = \boldsymbol{q}_0, k=0$
  2. $\Delta\boldsymbol{q} =$ solution of a QP that minimizes $||\mathcal{V}_b||$
  3. $\boldsymbol{q}_{k+1} = \boldsymbol{q}_k + \Delta\boldsymbol{q}$
  4. If $||\mathcal{V}_b||\approx 0$, we stop, otherwise $k=k+1$ and we go back to step 2

The QP looks like this:

$\min_{\boldsymbol{x}}f(\boldsymbol{x}) = \frac{1}{2}\boldsymbol{x}^T\boldsymbol{Q}\boldsymbol{x} + \boldsymbol{q}^T\boldsymbol{x}$

s.t. $\boldsymbol{A}\boldsymbol{x}-\boldsymbol{b} = \boldsymbol{0}$

and  $\boldsymbol{C}\boldsymbol{x}-\boldsymbol{d}\leq\boldsymbol{0}$

So we need to find what we should put in $\boldsymbol{Q}, \boldsymbol{q}, \boldsymbol{A}, \boldsymbol{b}, \boldsymbol{C}, \boldsymbol{d}$ and the optimization variables!

So:
- We have the desired end-effector "velocity" (error): $\mathcal{V}_b$
- We can compute the current velocity as: $\mathcal{V} = \boldsymbol{J}_b(\boldsymbol{q})\boldsymbol{v}$
- We need to find $\boldsymbol{v}_*$ such that $\mathcal{V}-\mathcal{V}_b=0$!
- We use for variables $\boldsymbol{x} = \boldsymbol{v}\in\mathbb{R}^n$!
- $\boldsymbol{Q} = \boldsymbol{J}_b^T\boldsymbol{J}_b\in\mathbb{R}^{n\times n}$
- $\boldsymbol{q} = -\boldsymbol{J}_b^T\mathcal{V}_b\in\mathbb{R}^n$
- We do not have any equality contraints
- $\boldsymbol{C} = \begin{bmatrix}dt & \cdots & \cdots & \cdots \\\cdots & dt & \cdots & \cdots\\ & & \ddots & \\\cdots & \cdots & -dt & \cdots\\\cdots & \cdots & \cdots & -dt\end{bmatrix}\in\mathbb{R}^{2n\times n}, \boldsymbol{d} = \begin{bmatrix}\boldsymbol{q}_{\text{max}}-\boldsymbol{q}_k\\\boldsymbol{q}_{\text{max}}-\boldsymbol{q}_k\\\vdots\\\boldsymbol{q}_k-\boldsymbol{q}_{\text{min}}\\\boldsymbol{q}_k-\boldsymbol{q}_{\text{min}}\end{bmatrix}\in\mathbb{R}^{2n}$

In [ ]:
import pinocchio as pin # for computing robot-related quantities
import example_robot_data # robot URDFs
import numpy as np # Linear algebra
import proxsuite # QP solver

In [ ]:
# Load the Z1 robot arm
z1 = example_robot_data.load("z1")

# Load the urdf model
model = z1.model

# Create data required by the algorithms
data = model.createData()

First, we need to create a method that computes the forward kinematics using the `Pinocchio` library.

In [ ]:
# Helpers
# function to compute FK for all frames/joints
def fk_all(model, data, q):
    pin.forwardKinematics(model,data,q) # FK
    pin.updateFramePlacements(model, data) # Update frames

In [ ]:
# Sample a random configuration as the target
pin.seed(847646089) # give a specific seed to pinocchio
qd = pin.randomConfiguration(model)
print('qd:', qd.T)

In [ ]:
# We want to compute IK for our tip ("link06")
frame_id = model.getFrameId("link06")

In [ ]:
# first of all, let's compute the transformation matrix
fk_all(model, data, qd)
T_wd = data.oMf[frame_id].copy()

print(T_wd)

In [ ]:
# Now let's create some useful functions
def compute_error(q_k):
    # We want to compute the error only in xyz position, aka we ignore the orientation
    # We should return the error as a 3D vector
    # This should be the local frame error
    ### ANSWER: Insert code here
    fk_all(model, data, q_k)
    Twb = data.oMf[frame_id]
    Vb_skew = pin.log6(Twb.actInv(T_wd))
    
    error = np.zeros((3,))
    error[0] = Vb_skew.vector[0]
    error[1] = Vb_skew.vector[1]
    error[2] = Vb_skew.vector[2]    
    ### END of ANSWER
    return error

def compute_jacobian(q_k):
    # We want a function that computes the 3xn Jacobian of a manipulator (aka we care only about xyz position)
    ### ANSWER: Insert code here
    fk_all(model, data, q_k)
    J = pin.computeFrameJacobian(model, data, q_k, frame_id)
    J = J[:3][:]
    ### END of ANSWER
    return J

def compute_QP_data(J, error, q_k, step):
    # We want to compute the QP matrices and vectors
    # In particular, we need to compute:
    #     - the matrix Q
    #     - the vector q
    #     - the matrix C
    #     - the vectors d_min, d_max
    # Please add a regularization of size 1e-6
    ### ANSWER: Insert code here
    Q = J.T @ J + 1e-6 * np.eye(model.nv)
    q = -J.T @ error
    C = step * np.eye(7,7)
    d_min = model.lowerPositionLimit - q_k
    d_max = model.upperPositionLimit - q_k
    ### END of ANSWER
    return Q, q, C, d_min, d_max

In [ ]:
pin.seed(1) # for reproducibility
qk = pin.randomConfiguration(model)

error = compute_error(qk)
assert(error.shape[0] == 3)
assert(np.isclose(error, np.array([0.091, -0.076, 0.508]), rtol=1e-3, atol=1e-3).all())

In [ ]:
pin.seed(1) # for reproducibility
qk = pin.randomConfiguration(model)

J = compute_jacobian(qk)
assert(J.shape[0] == 3)
assert(J.shape[1] == 7)
assert(np.isclose(J, np.array([[0.07844676, 0.04902833, 0.0926024, 0., 0., 0., 0.], [0.0380424, -0.141971, 0.2021557, 0.0913903, -0.00581146, 0., 0.], [0.04997564, -0.11545815, -0.16212844, 0.01087104, 0.04885557, 0., 0.]]), rtol=1e-3, atol=1e-3).all())

In [ ]:
pin.seed(20) # for reproducibility
qk = pin.randomConfiguration(model)

J = compute_jacobian(qk)
error = compute_error(qk)

Q, q, C, d_min, d_max = compute_QP_data(J, error, qk, 1.)
assert(Q.shape == (7, 7))
assert(q.shape[0] == 7)
assert(C.shape == (7, 7))
assert(d_min.shape[0] == 7)
assert(d_max.shape[0] == 7)


In [ ]:
np.set_printoptions(precision=3, suppress=True)
# IK via QP
error = np.ones((6, 1))
# We start without any guess, all zeros!
q_k = z1.q0.copy()

# Let's make a QP object
qp_dim = model.nv
qp_dim_eq = 0 # no equality constraint!
qp_dim_in = model.nv # joint limits. ProxSuite supports double limits: d_min <= Cx <= d_max!
qp = proxsuite.proxqp.dense.QP(qp_dim, qp_dim_eq, qp_dim_in)

step = 0.1
it = 0
success = False
while True:
    # We first compute the error
    error = compute_error(q_k)

    if (np.abs(error) <= 1e-6).all():
        success = True
        break
    if it > 10000:
        success = False
        break
    # We now need the Jacobian
    J = compute_jacobian(q_k)
    # And finally let's get the QP data
    Q, q, C, d_min, d_max = compute_QP_data(J, error, q_k, step)
    if it == 0: # in first iteration we initialize the model
        qp.init(Q, q, None, None, C, d_min, d_max)
    else: # otherwise, we update the model
        qp.update(Q, q, None, None, C, d_min, d_max)
    # Let's solve the QP
    qp.solve()
    # We get back the results
    v = np.copy(qp.results.x)
    # Compute next q_k given the velocity
    q_k = pin.integrate(model, q_k, v * step)
    it += 1
if success:
    print("Found solution in %d iterations with error: %s" % (it, error.T))
else:
    print("Could not find solution in %d iterations! Error: %s" % (it, error.T))
print(q_k.T)

In [ ]:
# Validation
fk_all(model, data, q_k)
for frame, oMf in zip(model.frames, data.oMf):
    if "link" not in frame.name:
        continue
    print(("{:<10} : {: .2f} {: .2f} {: .2f}"
          .format(frame.name, *oMf.translation.T.flat )))
print("================================")

# target configuration!
fk_all(model, data, qd)
for frame, oMf in zip(model.frames, data.oMf):
    if "link" not in frame.name:
        continue
    print(("{:<10} : {: .2f} {: .2f} {: .2f}"
          .format(frame.name, *oMf.translation.T.flat )))
print("================================")